<a href="https://colab.research.google.com/github/shiehn/dawnet-remotes/blob/main/DAWNet_Remote_Style_Transfer_MusicGen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

GPU CHECK

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

INSTALL MUSIC_GEN

In [ ]:
!python3 -m pip install -U git+https://github.com/facebookresearch/audiocraft#egg=audiocraft
# !python3 -m pip install -U audiocraft

import torchaudio
from audiocraft.models import MusicGen
from audiocraft.data.audio import audio_write

model = MusicGen.get_pretrained('facebook/musicgen-melody')



In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

ffmpeg_installed = !command -v ffmpeg
if not ffmpeg_installed:
    !apt-get install ffmpeg

!pip install dawnet-client

import librosa
import soundfile as sf

import uuid
import os
import dawnet_client.core as dawnet
from dawnet_client import DAWNetFilePath, utils

placeholder_txt = "Enter the token generated by the DAWNet plugin"
DAWNET_TOKEN = "5941402a-38d3-4064-bf23-6e0b44cafe0b" #@param {type:"string"}
dawnet_token = DAWNET_TOKEN

if dawnet_token is None or dawnet_token == "" or dawnet_token == placeholder_txt:
  print("ERROR: The token provided is not valid.")
  exit()

async def dawnet_func(description:str="childrens choir", audio_input:DAWNetFilePath=None):
    try:
        print("FILEPATH: " + str(audio_input))

        audio_length=utils.get_audio_length(audio_input)

        print("audio_length: " + str(audio_length))

        bpm_str = str(round(dawnet.get_daw_bpm()))
        print("bpm: " + bpm_str

        num_samples=1 # REPLACE ME
        output_audio_name = "output.wav"

        model.set_generation_params(duration=audio_length)
        descriptions = [description + " at " + bpm_str + " bpm"]
        melody, sr = torchaudio.load(audio_input)

        # generates using the melody from the given audio and the provided descriptions.
        wav = model.generate_with_chroma(descriptions, melody[None].expand(num_samples, -1, -1), sr)

        print('generations complete')

        for idx, one_wav in enumerate(wav):
          # Will save under {idx}.wav, with loudness normalization at -14 db LUFS.
          #audio_write(f'{idx}', one_wav.cpu(), model.sample_rate, strategy="loudness", loudness_compressor=True)

          # Move the tensor to CPU, convert to numpy array, reshape and ensure the data type is float32
          generated_audio = one_wav.cpu().numpy().astype('float32')
          generated_audio = generated_audio.squeeze()  # Remove extra dimensions

          output_audio_name = f"{str(uuid.uuid4())}.wav"
          sf.write(output_audio_name, generated_audio, 32000, subtype='PCM_16')  # Save as WAV file

          print("Adding file to output: " + str(output_audio_name))

          # after executing your custom code you send data back to the plugin like so ..
          await dawnet.output().add_file(output_audio_name)


        await dawnet.output().add_message("Returning the input file")
        await dawnet.output().send()

        print("output sent.")

        return True
    except Exception as e:
        print(f"Error in arbitrary_method: {e}")
        return f"Method encountered an error: {e}"


dawnet.set_token(token=dawnet_token)
dawnet.set_name("MusicGen Style Transfer")
dawnet.set_description("Provide a text description, number of samples, and sample lenth in seconds")
dawnet.register_method(dawnet_func)

dawnet.set_input_target_format('wav')
dawnet.set_input_target_channels(2)
dawnet.set_input_target_sample_rate(32000)
dawnet.set_input_target_bit_depth(16)

dawnet.set_output_target_format('wav')
dawnet.set_output_target_channels(2)
dawnet.set_output_target_sample_rate(44100)
dawnet.set_output_target_bit_depth(16)

dawnet.connect_to_server()

See our repo https://github.com/facebookresearch/audiocraft for more details on how to use this model!

See also [MusicGen Gradio Demo](https://colab.research.google.com/drive/1-Xe9NCdIs2sCUbiSmwHXozK6AAhMm7_i?usp=sharing) for a Colab using the Gradio app instead!
